In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import argparse
import tensorflow as tf
import random
import IPython.display as display
from IPython.display import clear_output
from random import randint

# Configuraciones de TensorFlow

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [ ]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

# Carga de Datos

In [2]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [3]:
batch_size = 30
size = (112,112)
frames = 16
canales = 3

In [4]:
import sys
sys.path.append('../')

from Datasets_utils.DatasetsLoader import VideoDataGenerator

dataset = VideoDataGenerator(root_path,batch_size,True)

In [8]:
import cv2
import os

path = dataset.videos_train_path[0]
for frame in sorted(os.listdir(path)):
    frame_path = os.path.join(path,frame)
    imagen = cv2.imread(frame_path)
    break
    
type(imagen)

numpy.ndarray

# Red Neuronal

In [ ]:
from LTC import LTC

Sesion = tf.Session(config=config)

ltc = LTC(101,batch_size,[16,112,112,3])

ltc.enable_training()

In [ ]:
ltc.inicializar_modelo(0.003,0.5)
init = tf.global_variables_initializer()
Sesion.run(init)

In [ ]:
#Como es primera ver corriendo este modelo entonces no debo cargar nada
saver = tf.train.Saver()
steps = 160000
epoch = 0
for paso in range(steps):
    clear_output()
    
    #Primero voy salvando el modelo de 40000 en 40000
    if paso % 40000 == 0:
        saved_path = saver.save(Sesion, '../Checkpoints/LTC_112x112x16_rgb')
        print('Modelo salvado en: {}'.format(saved_path))
        
    if paso == 80000 or paso == 125000:
        ltc.lr = ltc.lr * 0.1
    
    train_batch, train_labels = dataset.get_next_train_batch(size,frames,canales)
    #test_batch, test_labels = dataset.get_next_test_batch(size,frames,canales)
    train_batch = Sesion.run(train_batch.eval(session=Sesion))
    train_labels = Sesion.run(train_labels.eval(session=Sesion))
    
    if dataset.train_batch_index == 0:
        epoch += 1
        
    print("Epoch: ",epoch)
    print("Train batch: ",dataset.train_batch_index+1,"/",dataset.train_batches)
    
    #Propago
    Sesion.run(ltc.entrenar,feed_dict={ltc.x: train_batch, ltc.y: train_labels})
    print("Loss: ",ltc.perdidas[-1]," Acuraccy: ",ltc.precisiones[-1])
    
print("Final de entrenamiento")

In [ ]:
dataset.